# Import Libraries

In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import time
import random
random.seed(42)
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 디바이스 선택 코드
if torch.cuda.is_available():
    # CUDA(GPU) 사용 가능 여부 확인
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    # MPS(Metal Performance Shaders) 사용 가능 여부 확인
    device = torch.device("mps")
else:
    # CUDA와 MPS 모두 없으면 CPU 사용
    device = torch.device("cpu")

# Data Load

In [2]:
# 파일 경로 설정
business_path = 'data/yelp_academic_dataset_business.json'

# JSON 파일에서 일부 데이터만 읽기
n_lines = 3  # 읽고자 하는 줄 수

# 줄 단위로 읽고 필요한 만큼만 데이터프레임으로 변환
with open(business_path, 'r') as file:
    # 필요한 줄 수만큼 JSON 읽기
    lines = [file.readline() for _ in range(n_lines)]

# 읽은 줄을 DataFrame으로 변환
business = pd.DataFrame([pd.read_json(line, lines=False, typ='series') for line in lines])

# 결과 확인
business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


In [3]:
# 파일 경로 설정
checkin_path = 'data/yelp_academic_dataset_checkin.json'

# JSON 파일에서 일부 데이터만 읽기
n_lines = 3  # 읽고자 하는 줄 수

# 줄 단위로 읽고 필요한 만큼만 데이터프레임으로 변환
with open(checkin_path, 'r') as file:
    # 필요한 줄 수만큼 JSON 읽기
    lines = [file.readline() for _ in range(n_lines)]

# 읽은 줄을 DataFrame으로 변환
checkin = pd.DataFrame([pd.read_json(line, lines=False, typ='series') for line in lines])

# 결과 확인
checkin.head(3)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"


In [4]:
# 파일 경로 설정
review_path = 'data/yelp_academic_dataset_review.json'

# JSON 파일에서 일부 데이터만 읽기
n_lines = 3  # 읽고자 하는 줄 수

# 줄 단위로 읽고 필요한 만큼만 데이터프레임으로 변환
with open(review_path, 'r') as file:
    # 필요한 줄 수만큼 JSON 읽기
    lines = [file.readline() for _ in range(n_lines)]

# 읽은 줄을 DataFrame으로 변환
review = pd.DataFrame([pd.read_json(line, lines=False, typ='series') for line in lines])

# 결과 확인
review.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


In [5]:
# 파일 경로 설정
tip_path = 'data/yelp_academic_dataset_tip.json'

# JSON 파일에서 일부 데이터만 읽기
n_lines = 3  # 읽고자 하는 줄 수

# 줄 단위로 읽고 필요한 만큼만 데이터프레임으로 변환
with open(tip_path, 'r') as file:
    # 필요한 줄 수만큼 JSON 읽기
    lines = [file.readline() for _ in range(n_lines)]

# 읽은 줄을 DataFrame으로 변환
tip = pd.DataFrame([pd.read_json(line, lines=False, typ='series') for line in lines])

# 결과 확인
tip.head(3)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0


In [6]:
# 파일 경로 설정
user_path = 'data/yelp_academic_dataset_user.json'

# JSON 파일에서 일부 데이터만 읽기
n_lines = 3  # 읽고자 하는 줄 수

# 줄 단위로 읽고 필요한 만큼만 데이터프레임으로 변환
with open(user_path, 'r') as file:
    # 필요한 줄 수만큼 JSON 읽기
    lines = [file.readline() for _ in range(n_lines)]

# 읽은 줄을 DataFrame으로 변환
user = pd.DataFrame([pd.read_json(line, lines=False, typ='series') for line in lines])

# 결과 확인
user.head(3)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18


# Data Pre-processing

- index | user_id | business_id | stars | text | year

- Filtering
    - City : Philadephia
    - review count : 5>=

In [7]:
# 파일 경로 설정
business_path = 'data/yelp_academic_dataset_business.json'

# JSON 파일 전체를 불러오되, 필요한 데이터만 추출
city_filter = "Philadelphia"  # 필터링할 도시 이름
min_reviews = 5  # 최소 리뷰 수

# 결과 저장할 빈 DataFrame 초기화
filtered_data = pd.DataFrame()

# 청크 단위로 JSON 파일 읽기
for chunk in pd.read_json(business_path, lines=True, chunksize=10000):
    # 'city' 컬럼이 "Philadelphia"이고 'review_count'가 5 이상인 데이터만 필터링
    filtered_chunk = chunk[(chunk['city'] == city_filter) & (chunk['review_count'] >= min_reviews)]
    # 결과를 filtered_data에 추가
    filtered_data = pd.concat([filtered_data, filtered_chunk], ignore_index=True)

filtered_data.tail(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
14566,9U1Igcpe954LoWZRmNc-zg,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24",Philadelphia,PA,19147,39.932756,-75.144504,3.0,32,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Day Spas, Beauty & Spas, Skin Care, Massage","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
14567,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
14568,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [8]:
business_ids = filtered_data[['business_id']]

# JSON 파일 전체를 처리하면서 동일한 business_id만 필터링
review_path = 'data/yelp_academic_dataset_review.json'

# 결과 저장할 빈 DataFrame 초기화
filtered_reviews = pd.DataFrame()

# 청크 단위로 JSON 파일 읽기
for chunk in pd.read_json(review_path, lines=True, chunksize=10000):
    # business_id 기준으로 필터링
    filtered_chunk = chunk[chunk['business_id'].isin(business_ids['business_id'])]
    # 결과를 filtered_reviews에 추가
    filtered_reviews = pd.concat([filtered_reviews, filtered_chunk], ignore_index=True)
    
filtered_reviews.tail(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
967549,99ylx-qPUSseITqBye2MpA,-AkziDwQ8hv2COTDBBUpig,aunmz06iWvo3bd6MMHEbqg,3,0,0,0,Philly has become a dangerous place with the m...,2022-01-18 03:48:44
967550,KlHxcAifUF5zDKpJCBrRsw,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4,1,0,0,"ice cream! ice cream sodas, sundaes!! \n\nwant...",2012-10-21 04:08:40
967551,cACxcUY_AIsQKkpDRXuqnw,MCzlzlOw7IGbRAKVjJBPtg,fcGexL5VH5G2Xw0tRj9uOQ,3,1,1,0,This is a good pizza option - they deliver thr...,2018-03-13 13:54:48


In [9]:
data = filtered_reviews[['user_id', 'business_id', 'stars', 'text', 'date']]
# date 컬럼에서 연도만 추출
data['year'] = data['date'].dt.year

# 기존 date 컬럼 이름을 year로 변경
data = data.drop(columns=['date'])  # 기존 date 컬럼 삭제
data.tail(3)

,user_id,business_id,stars,text,year
967549,-AkziDwQ8hv2COTDBBUpig,aunmz06iWvo3bd6MMHEbqg,3,Philly has become a dangerous place with the m...,2022
967550,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4,"ice cream! ice cream sodas, sundaes!! \n\nwant...",2012
967551,MCzlzlOw7IGbRAKVjJBPtg,fcGexL5VH5G2Xw0tRj9uOQ,3,This is a good pizza option - they deliver thr...,2018


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967552 entries, 0 to 967551
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      967552 non-null  object
 1   business_id  967552 non-null  object
 2   stars        967552 non-null  int64 
 3   text         967552 non-null  object
 4   year         967552 non-null  int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 33.2+ MB


- 모델의 입력으로 사용하기 위해, 문자열로 이루어진 유저ID와 레스토랑ID를 각각 고유한 정수로 인코딩

In [11]:
from sklearn.preprocessing import LabelEncoder
 
# 유저와 레스토랑을 고유한 정수로 인코딩
le = LabelEncoder()
data['user_id'] = le.fit_transform(data['user_id'])
data['business_id'] = le.fit_transform(data['business_id'])
 
# 필요없는 열 제거
data.drop(['text', 'year'], axis=1, inplace=True)
data.head()
 
# 총 유저, 레스토랑 수
num_user = data['user_id'].nunique()
num_res = data['business_id'].nunique()
 
print('전체 유저 수 : {0} / 아이템 수 : {1}'.format(num_user, num_res))


전체 유저 수 : 279857 / 아이템 수 : 14569


## Leave one out
- 데이터셋 분리

In [12]:
data['stars'] = 1
# 본 데이터에는 timestamp가 없기 때문에, user_index로 만들어준다.
data['timestamp'] = data['user_id'].index

# 유저별로 groupby 한 후에, 가장 첫번째 데이터를 test 세트로 분리한다.
data['rank_latest'] = data.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)

train_ratings = data[data['rank_latest'] != 1]
test_ratings = data[data['rank_latest'] == 1]

train_ratings = train_ratings[['user_id', 'business_id', 'stars']]
test_ratings = test_ratings[['user_id', 'business_id', 'stars']]

print('훈련 데이터셋 유저 수:', train_ratings['user_id'].nunique())
print('테스트 데이터셋 유저 수:', test_ratings['user_id'].nunique())

훈련 데이터셋 유저 수: 113881
테스트 데이터셋 유저 수: 279857


## Negative sampling
- 해당 데이터에는 구매여부(0 or 1)를 예측하지만, 데이터 세트에는 구매한 1 데이터 밖에 존재하지 않는다.
- 따라서, 구매하지 않은 데이터(0)을 만들어주기 위해, 전체 아이템(레스토랑) 풀에서 랜덤하게 생성한다.

In [13]:
# 전체 레스토랑 풀
all_rest = data['business_id'].unique()

# 빈 리스트 생성
users, items, labels= [], [], []

# 훈련 세트에 존재하는 유저-레스토랑 페어
user_item_set = set(zip(train_ratings['user_id'], train_ratings['business_id']))

# 4:1 비율로, 구매하지 않은 0 라벨을 만들어준다.
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive

    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_rest)
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_rest)
        users.append(u)
        items.append(negative_item)
        labels.append(0)

users = np.array(users)
items = np.array(items)
labels = np.array(labels)

train_df = pd.DataFrame({'user_id':users, 'business_id':items, 'labels':labels})
train_df.tail(3)

  0%|          | 0/661253 [00:00<?, ?it/s]

,user_id,business_id,labels
3306262,38372,10658,0
3306263,38372,5470,0
3306264,38372,7089,0


In [14]:
# 테스트 데이터는 negative sampling을 적용하지 않고, 나중에 Dataloader에서 배치를 적용할 예정
test_df = test_ratings.rename({'stars':'labels'}, axis=1)
test_df = test_df.reset_index().drop('index', axis=1)

# Dataset & Dataloader

In [15]:
class Rating_Datset(torch.utils.data.Dataset):
	def __init__(self, dataset):
		self.user = dataset['user_id']
		self.item = dataset['business_id']
		self.label = dataset['labels']

	def __len__(self):
		return len(self.user)

	def __getitem__(self, idx):
		u = self.user[idx]
		i = self.item[idx]
		l = self.label[idx]

		return torch.tensor(u), torch.tensor(i), torch.tensor(l)

train_dataset = Rating_Datset(train_df)
test_dataset = Rating_Datset(test_df)

train_dataloader = DataLoader(train_dataset, batch_size=256)
test_dataloader = DataLoader(test_dataset, batch_size=99)

# Define Model

![alt text](img/model_framework.png)

- 간단히 유저의 수와 레스토랑 수로 이루어진 임베딩을 구현한다. 이때 임베딩 차원을 8로 지정한다.
- 이후, [32 -> 16 -> 8] 크기의 레이어 3개를 통과하고, 구매 여부[0,1]를 예측하기 때문에 마지막 활성화 함수는 sigmoid를 적용한다.

In [16]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items):
        super(NCF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embed_dim = 16
        self.relu=nn.ReLU()
        self.sigmoid= nn.Sigmoid()
 
        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embed_dim)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embed_dim)
 
        self.fc1 = nn.Linear(in_features=32, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=8)
        self.fc3 = nn.Linear(in_features=8, out_features=1)

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector

        x = self.fc1(vector)
        x = self.relu(x)
        # x = nn.BatchNorm1d()(x)
        # x = nn.Dropout(p=0.1)(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        out = self.sigmoid(x)

        return out.squeeze()

num_users = data['user_id'].nunique()+1
num_items = data['business_id'].nunique()+1

model = NCF(num_users, num_items)
model.to(device)

NCF(
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (embedding_user): Embedding(279858, 16)
  (embedding_item): Embedding(14570, 16)
  (fc1): Linear(in_features=32, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
)

# Hit ratio & NDCG
- Hit ratio : 추천된 상위 K개의 아이템 중 실제로 사용자가 선호하는 아이템이 얼마나 포함되었는지를 측정한다.
- NDCG(Normalized Discounted Cumulative Gain) : 추천된 상위 K개의 아이템에 대해, 사용자의 관심도에 따라 순위를 매기고 할인된 누적 이득을 정규화한 값으로, 더 높은 NDCG 값은 더 좋은 추천을 의미한다.

In [17]:
def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0

def ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = pred_items.index(gt_item)
        return np.reciprocal(np.log2(index + 2))
    return 0

def metrics(model, test_loader, top_k):
    HR, NDCG = [], []

    for user, item, _ in test_loader:
        user = user.to(device)
        item = item.to(device)

        predictions = model(user, item)
        # 가장 높은 top_k개 선택
        _, indices = torch.topk(predictions, top_k)
        # 해당 상품 index 선택
        recommends = torch.take(item, indices).cpu().numpy().tolist()
        # 정답값 선택
        gt_item = item[0].item()
        HR.append(hit(gt_item, recommends))
        NDCG.append(ndcg(gt_item, recommends))

    return np.mean(HR), np.mean(NDCG)

# Model

In [19]:
learning_rate = 0.001
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

from tqdm.auto import tqdm

size = len(train_dataloader.dataset)
count, best_hr = 0, 0
top_k= 30

for epoch in range(10):
    print('##### EPOCH {} #####'.format(epoch + 1))

    model.train()  # Enable dropout (if have).
    start_time = time.time()

    for batch, (user, item, label) in enumerate(tqdm(train_dataloader)):
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)

        # gradient 초기화
        model.zero_grad()
        prediction = model(user, item)
        loss = loss_fn(prediction.to(torch.float32), label.to(torch.float32))
        loss.backward()
        optimizer.step()
        count += 1

    model.eval()
    HR, NDCG = metrics(model, test_dataloader, top_k)

    elapsed_time = time.time() - start_time

    print(
        "The time elapse of epoch {:03d}".format(epoch)
        + " is: "
        + time.strftime("%H: %M: %S", time.gmtime(elapsed_time))
    )
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch

print(
    "End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
        best_epoch, best_hr, best_ndcg
    )
)

##### EPOCH 1 #####


  0%|          | 0/12916 [00:00<?, ?it/s]

The time elapse of epoch 000 is: 00: 01: 57
HR: 0.329	NDCG: 0.127
##### EPOCH 2 #####


  0%|          | 0/12916 [00:00<?, ?it/s]

The time elapse of epoch 001 is: 00: 01: 57
HR: 0.333	NDCG: 0.128
##### EPOCH 3 #####


...

The time elapse of epoch 007 is: 00: 02: 13
HR: 0.352	NDCG: 0.131
##### EPOCH 9 #####


  0%|          | 0/12916 [00:00<?, ?it/s]

The time elapse of epoch 008 is: 00: 02: 04
HR: 0.354	NDCG: 0.132
##### EPOCH 10 #####


  0%|          | 0/12916 [00:00<?, ?it/s]

The time elapse of epoch 009 is: 00: 02: 06
HR: 0.354	NDCG: 0.131
End. Best epoch 008: HR = 0.354, NDCG = 0.132
